In [1]:
!pip install wget
import wget

# Download the dataset
url = 'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz'
wget.download(url)

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a0df654c033831659ccd7b20b0721fdca89826ed1e95e3b3d5ac103f232c1716
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


'lenta-ru-news.csv.gz'

In [2]:
import pandas as pd
import gzip

path = 'lenta-ru-news.csv.gz'

df = pd.read_csv(
    gzip.open(path, 'rt', encoding='utf-8'),
    delimiter=',',
    quotechar='"'
)

df = df[['title', 'text', 'topic']].head(50000)

class_counts = df['topic'].value_counts()
valid_classes = class_counts[class_counts > 500].index.to_list()
df = df[df['topic'].isin(valid_classes)]

In [3]:
!pip install natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 92.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=74ea96b974a29b58e70b80ad91fe11e1e84742b171d5c831c3933ea92671ee55
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=dcd3ab88bee626a463ba3e97502cbd154fdfc8474aa970d34077a97ad3670fda
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built docopt intervaltree


In [4]:
import re
from natasha import Doc, Segmenter, MorphVocab, NewsMorphTagger, NewsEmbedding
emb = NewsEmbedding()
morph_vocab = MorphVocab()
segmenter = Segmenter(emb)
morph_tagger = NewsMorphTagger(emb)

import re

def normalize(text):
    if not isinstance(text, str) or pd.isna(text) or text == "":
        return ""

    try:
        # Приведение к нижнему регистру
        text_lower = text.lower()

        # Удаление URL
        clean_text = re.sub(r'https?://\S+|www\.\S+', '', text_lower)

        # Удаление email
        clean_text = re.sub(r'\S+@\S+', '', clean_text)

        # Удаление всех чисел
        clean_text = re.sub(r'\d+', '', clean_text)

        # Удаление пунктуации
        clean_text = re.sub(r'[^\w\s]', '', clean_text)

        # Удаление множественных пробелов
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()

        # Создание документа
        doc = Doc(clean_text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)

        lemmatized_tokens = []
        for token in doc.tokens:
            try:
                token.lemmatize(morph_vocab)
                if hasattr(token, 'lemma') and token.lemma:
                    lemmatized_tokens.append(token.lemma)
                else:
                    lemmatized_tokens.append(token.text)
            except AttributeError:
                lemmatized_tokens.append(token.text)

        # Собираем результат
        lemmatized_text = ' '.join(lemmatized_tokens)
        return lemmatized_text

    except Exception as e:
        print(f"Ошибка при обработке текста: {str(e)}")
        return clean_text

In [5]:
df['normalized_text'] = df['text'].apply(normalize)
df['normalized_title'] = df['title'].apply(normalize)

In [6]:
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split

dataset = df[['normalized_text', 'normalized_title', 'topic']]

label_encoder = LabelEncoder()
encoded_topics = label_encoder.fit_transform(dataset['topic'])

y = encoded_topics
X = dataset['normalized_title'] + ' ' + dataset['normalized_text']

X_train, X_prep, y_train, y_prep  = train_test_split(X, y, test_size=0.4, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_prep, y_prep, test_size=0.5, random_state=1)

X

,0
0,назвать регион россия с сам высокий смертность...
1,австрия не представить доказательство вина рос...
2,обнаружить самый счастливый место на планета с...
3,в сша раскрыть сумма расход на расследование р...
4,хакер рассказать о план великобритания заминир...
...,...
49995,в великобритания арестовать мужчина за секс с ...
49996,создать эффективный способ лечение смертоносны...
49997,защитник сборная россия спасти ворота кельн в ...
49998,летний хипстерполицейский покорить instagram и...


In [7]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [8]:
tokenized_texts = []
for text in X_train:
    tokens = text.split()
    tokenized_texts.append(tokens)

# Обучаем модель word2vec
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=300,  # размерность векторов
    window=7,         # размер окна контекста
    min_count=5,      # минимальная частота слова
    workers=4,        # число потоков для параллельного обучения
    sg=1,             # 1 = skip-gram; 0 = CBOW
    seed=1,
    epochs=10,
    compute_loss=True,
)

training_loss = w2v_model.get_latest_training_loss()
print(training_loss)

67934632.0


Сначала обучил на дефолтных параметрах затем увеличил размер окна, добавил больше эпох и сменил на скип грам

In [9]:
w2v_model.save("w2v_model.model")

In [10]:
word_vectors = w2v_model.wv

word_vectors.doesnt_match("завтрак тарелка ужин обед".split())

'тарелка'

In [11]:
result = word_vectors.most_similar(positive=['женщина', 'король'], negative=['мужчина'])
most_similar_key, similarity = result[0]  # look at the first match
print(f"{most_similar_key}: {similarity:.4f}")
queen: 0.7699

бритт: 0.4519


In [12]:
import urllib.request
import gensim

urllib.request.urlretrieve(
    "https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz",
    "ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz"
)

model_path = 'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
model_rusvectores = gensim.models.KeyedVectors.load_word2vec_format(model_path)

In [13]:
!pip install navec
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
from navec import Navec
path = 'navec_news_v1_1B_250K_300d_100q.tar'
model_navec = Navec.load(path)
model_navec['человек'][:15]

--2025-03-14 05:59:34--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  7.78MB/s    in 4.8s    

2025-03-14 05:59:40 (5.24 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]



array([-0.13068067, -0.12051002, -0.05782367,  0.07967507,  0.08338855,
        0.59920526,  0.4020081 , -1.0838276 ,  0.12556174,  0.17060532,
        0.16637331, -0.00257014,  0.51296437,  0.17175263, -0.40394753],
      dtype=float32)

In [14]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Функция для получения усредненного вектора текста
def get_text_vector(text, model, size=300):
    tokens = text.split()
    vectors = []

    for token in tokens:
        if token in model:  # Проверяем, есть ли слово в модели
            vectors.append(model[token])

    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(size)  # Если ни одно слово не найдено, возвращаем нулевой вектор

# Создаем векторные представления для каждой модели
# 1. Наша обученная модель
X_train_w2v = np.array([get_text_vector(text, w2v_model.wv) for text in X_train])
X_val_w2v = np.array([get_text_vector(text, w2v_model.wv) for text in X_val])
X_test_w2v = np.array([get_text_vector(text, w2v_model.wv) for text in X_test])

# 2. RusVectores
X_train_rusvec = np.array([get_text_vector(text, model_rusvectores) for text in X_train])
X_val_rusvec = np.array([get_text_vector(text, model_rusvectores) for text in X_val])
X_test_rusvec = np.array([get_text_vector(text, model_rusvectores) for text in X_test])

# 3. Navec
X_train_navec = np.array([get_text_vector(text, model_navec) for text in X_train])
X_val_navec = np.array([get_text_vector(text, model_navec) for text in X_val])
X_test_navec = np.array([get_text_vector(text, model_navec) for text in X_test])

In [15]:
# Преобразуем метки классов
from sklearn.preprocessing import LabelEncoder

# Обучение логистической регрессии для каждой модели эмбеддингов
# 1. Для нашей модели Word2Vec
lr_w2v = LogisticRegression(max_iter=1000, C=1.0, random_state=1)
lr_w2v.fit(X_train_w2v, y_train)

# 2. Для RusVectores
lr_rusvec = LogisticRegression(max_iter=1000, C=1.0, random_state=1)
lr_rusvec.fit(X_train_rusvec, y_train)

# 3. Для Navec
lr_navec = LogisticRegression(max_iter=1000, C=1.0, random_state=1)
lr_navec.fit(X_train_navec, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [16]:
# Оценка на валидационной выборке
# 1. Для нашей модели Word2Vec
y_val_pred_w2v = lr_w2v.predict(X_val_w2v)
val_accuracy_w2v = accuracy_score(y_val, y_val_pred_w2v)
print(f"Accuracy на валидационной выборке (наша Word2Vec): {val_accuracy_w2v:.4f}")
print(classification_report(y_val, y_val_pred_w2v))

# 2. Для RusVectores
y_val_pred_rusvec = lr_rusvec.predict(X_val_rusvec)
val_accuracy_rusvec = accuracy_score(y_val, y_val_pred_rusvec)
print(f"Accuracy на валидационной выборке (RusVectores): {val_accuracy_rusvec:.4f}")
print(classification_report(y_val, y_val_pred_rusvec))

# 3. Для Navec
y_val_pred_navec = lr_navec.predict(X_val_navec)
val_accuracy_navec = accuracy_score(y_val, y_val_pred_navec)
print(f"Accuracy на валидационной выборке (Navec): {val_accuracy_navec:.4f}")
print(classification_report(y_val, y_val_pred_navec))

Accuracy на валидационной выборке (наша Word2Vec): 0.8345
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       610
           1       0.85      0.78      0.82       408
           2       0.78      0.83      0.80       682
           3       0.79      0.78      0.78       772
           4       0.90      0.88      0.89       667
           5       0.81      0.87      0.84      1342
           6       0.89      0.86      0.88       701
           7       0.80      0.64      0.71       238
           8       0.74      0.78      0.76      1381
           9       0.79      0.74      0.76       536
          10       0.96      0.97      0.96      1129
          11       0.90      0.83      0.86       427
          12       0.84      0.84      0.84       975

    accuracy                           0.83      9868
   macro avg       0.84      0.82      0.83      9868
weighted avg       0.84      0.83      0.83      9868

Accuracy на валидацио

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X_train)
feature_names = tfidf_vectorizer.get_feature_names_out()

In [18]:
tfidf_vector = tfidf_vectorizer.transform(['называть регион россия с самой высокий смертность'])

nonzero_indices = tfidf_vector.nonzero()[1]
nonzero_indices
tfidf_weights = tfidf_vector.data
words = [feature_names[idx] for idx in nonzero_indices]

for word, weight in zip(words, tfidf_weights):
    print(f"{word}: {weight}")

высокий: 0.3879544967330891
называть: 0.40634769876295623
регион: 0.3641314121990146
россия: 0.19894026947032442
смертность: 0.7156842460244959


In [19]:
print("Оригинал:", df['title'][0])
print("Нормализованный:", df['normalized_title'][0])

Оригинал: Названы регионы России с самой высокой смертностью от рака
Нормализованный: назвать регион россия с сам высокий смертность от рак


In [20]:


# 2. Функция для создания TF-IDF взвешенных векторов
def get_tfidf_weighted_vector(text, word_vectors, vector_size=300):
    tfidf_vector = tfidf_vectorizer.transform([text])

    nonzero_indices = tfidf_vector.nonzero()[1]
    tfidf_weights = tfidf_vector.data
    words = [feature_names[idx] for idx in nonzero_indices]

    # Вычисляем взвешенную сумму векторов слов
    weighted_sum = np.zeros(vector_size)
    total_weight = 0

    for word, weight in zip(words, tfidf_weights):
        if word in word_vectors:
            weighted_sum += word_vectors[word] * weight
            total_weight += weight

    # Нормализуем по сумме весов
    if total_weight > 0:
        return weighted_sum / total_weight
    else:
        # Если не нашли слов с TF-IDF весами, используем обычное усреднение
        tokens = text.split()
        found_vectors = [word_vectors[token] for token in tokens if token in word_vectors]
        if found_vectors:
            return np.mean(found_vectors, axis=0)
        else:
            return np.zeros(vector_size)

# 3. Создаем TF-IDF взвешенные векторы для наборов данных
X_train_tfidf_weighted = np.array([get_tfidf_weighted_vector(text, w2v_model.wv) for text in X_train])
X_val_tfidf_weighted = np.array([get_tfidf_weighted_vector(text, w2v_model.wv) for text in X_val])
X_train_tfidf_weighted

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 5. Обучаем логистическую регрессию на TF-IDF взвешенных векторах
lr_tfidf_weighted = LogisticRegression(max_iter=1000, C=1.0, random_state=1)
lr_tfidf_weighted.fit(X_train_tfidf_weighted, y_train_encoded)

# 6. Оцениваем модель на валидационной выборке
y_val_pred = lr_tfidf_weighted.predict(X_val_tfidf_weighted)
val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
print(f"Accuracy на валидационной выборке (Word2Vec с TF-IDF взвешиванием): {val_accuracy:.4f}")
print(classification_report(y_val_encoded, y_val_pred))

Accuracy на валидационной выборке (Word2Vec с TF-IDF взвешиванием): 0.8359
              precision    recall  f1-score   support

           0       0.82      0.81      0.82       610
           1       0.86      0.77      0.82       408
           2       0.77      0.83      0.80       682
           3       0.79      0.80      0.79       772
           4       0.91      0.88      0.89       667
           5       0.82      0.86      0.84      1342
           6       0.89      0.86      0.88       701
           7       0.80      0.69      0.74       238
           8       0.75      0.78      0.76      1381
           9       0.79      0.72      0.76       536
          10       0.97      0.97      0.97      1129
          11       0.90      0.84      0.87       427
          12       0.84      0.85      0.84       975

    accuracy                           0.84      9868
   macro avg       0.84      0.82      0.83      9868
weighted avg       0.84      0.84      0.84      9868



In [21]:
# Оценка на валидационной выборке
# 1. Для нашей модели Word2Vec
y_val_pred_w2v = lr_w2v.predict(X_val_w2v)
val_accuracy_w2v = accuracy_score(y_val, y_val_pred_w2v)
print(f"Accuracy на валидационной выборке (наша Word2Vec): {val_accuracy_w2v:.4f}")
print(classification_report(y_val, y_val_pred_w2v))

# 2. Для RusVectores
y_val_pred_rusvec = lr_rusvec.predict(X_val_rusvec)
val_accuracy_rusvec = accuracy_score(y_val, y_val_pred_rusvec)
print(f"Accuracy на валидационной выборке (RusVectores): {val_accuracy_rusvec:.4f}")
print(classification_report(y_val, y_val_pred_rusvec))

# 3. Для Navec
y_val_pred_navec = lr_navec.predict(X_val_navec)
val_accuracy_navec = accuracy_score(y_val, y_val_pred_navec)
print(f"Accuracy на валидационной выборке (Navec): {val_accuracy_navec:.4f}")
print(classification_report(y_val, y_val_pred_navec))

Accuracy на валидационной выборке (наша Word2Vec): 0.8345
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       610
           1       0.85      0.78      0.82       408
           2       0.78      0.83      0.80       682
           3       0.79      0.78      0.78       772
           4       0.90      0.88      0.89       667
           5       0.81      0.87      0.84      1342
           6       0.89      0.86      0.88       701
           7       0.80      0.64      0.71       238
           8       0.74      0.78      0.76      1381
           9       0.79      0.74      0.76       536
          10       0.96      0.97      0.96      1129
          11       0.90      0.83      0.86       427
          12       0.84      0.84      0.84       975

    accuracy                           0.83      9868
   macro avg       0.84      0.82      0.83      9868
weighted avg       0.84      0.83      0.83      9868

Accuracy на валидацио

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
# Подготовим TF-IDF взвешенные векторы
X_train_tfidf_weighted = np.array([get_tfidf_weighted_vector(text, word_vectors, 300) for text in X_train])
X_test_tfidf_weighted = np.array([get_tfidf_weighted_vector(text, word_vectors, 300) for text in X_test])

# Обучим логистическую регрессию на TF-IDF взвешенных векторах
lr_tfidf_weighted = LogisticRegression(max_iter=1000, C=1.0, random_state=1)
lr_tfidf_weighted.fit(X_train_tfidf_weighted, y_train)

# ФИНАЛЬНОЕ СРАВНЕНИЕ НА ТЕСТОВОЙ ВЫБОРКЕ
# 1. Для нашей модели Word2Vec
y_test_pred_w2v = lr_w2v.predict(X_test_w2v)
test_accuracy_w2v = accuracy_score(y_test, y_test_pred_w2v)
print(f"Accuracy на тестовой выборке (наша Word2Vec): {test_accuracy_w2v:.4f}")
print(classification_report(y_test, y_test_pred_w2v))

# 2. Для RusVectores
y_test_pred_rusvec = lr_rusvec.predict(X_test_rusvec)
test_accuracy_rusvec = accuracy_score(y_test, y_test_pred_rusvec)
print(f"Accuracy на тестовой выборке (RusVectores): {test_accuracy_rusvec:.4f}")
print(classification_report(y_test, y_test_pred_rusvec))

# 3. Для Navec
y_test_pred_navec = lr_navec.predict(X_test_navec)
test_accuracy_navec = accuracy_score(y_test, y_test_pred_navec)
print(f"Accuracy на тестовой выборке (Navec): {test_accuracy_navec:.4f}")
print(classification_report(y_test, y_test_pred_navec))

# 4. Для Word2Vec с TF-IDF взвешиванием
y_test_pred_tfidf = lr_tfidf_weighted.predict(X_test_tfidf_weighted)
test_accuracy_tfidf = accuracy_score(y_test, y_test_pred_tfidf)
print(f"Accuracy на тестовой выборке (Word2Vec с TF-IDF взвешиванием): {test_accuracy_tfidf:.4f}")
print(classification_report(y_test, y_test_pred_tfidf))

# Сравнение всех моделей
print("\nСравнение accuracy всех моделей на тестовой выборке:")
print(f"Word2Vec: {test_accuracy_w2v:.4f}")
print(f"RusVectores: {test_accuracy_rusvec:.4f}")
print(f"Navec: {test_accuracy_navec:.4f}")
print(f"Word2Vec с TF-IDF взвешиванием: {test_accuracy_tfidf:.4f}")

# Определим лучшую модель
best_accuracy = max(test_accuracy_w2v, test_accuracy_rusvec, test_accuracy_navec, test_accuracy_tfidf)
if best_accuracy == test_accuracy_w2v:
    print("\nЛучшая модель: Word2Vec")
elif best_accuracy == test_accuracy_rusvec:
    print("\nЛучшая модель: RusVectores")
elif best_accuracy == test_accuracy_navec:
    print("\nЛучшая модель: Navec")
else:
    print("\nЛучшая модель: Word2Vec с TF-IDF взвешиванием")

Accuracy на тестовой выборке (наша Word2Vec): 0.8370
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       655
           1       0.85      0.82      0.83       397
           2       0.77      0.83      0.80       614
           3       0.79      0.75      0.77       767
           4       0.92      0.89      0.91       650
           5       0.83      0.87      0.85      1380
           6       0.91      0.89      0.90       738
           7       0.77      0.65      0.70       243
           8       0.75      0.78      0.77      1428
           9       0.76      0.73      0.74       539
          10       0.95      0.96      0.95      1029
          11       0.90      0.85      0.88       406
          12       0.84      0.85      0.84      1021

    accuracy                           0.84      9867
   macro avg       0.84      0.82      0.83      9867
weighted avg       0.84      0.84      0.84      9867

Accuracy на тестовой выбор

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
